In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import tree, metrics
from google.colab import drive
import pickle
import joblib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import pickle
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from scipy.stats import randint, uniform




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
males_legacy_df = pd.read_csv('/content/drive/My Drive/male_players.csv', low_memory = False)

In [ ]:
players_22_df = pd.read_csv('/content/drive/MyDrive/players_22.csv',low_memory = False)

In [ ]:
males_legacy_df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [ ]:
# Dropping all the duplicate data in the players_21 dataset
males_legacy_df = males_legacy_df.drop_duplicates()

# Drop columns with 30% or more missing values
threshold = 0.3
males_legacy_df = males_legacy_df.dropna(axis=1, thresh=int(threshold * len(males_legacy_df)))

# Drop columns containing "url"
males_legacy_df = males_legacy_df.drop(columns=[col for col in males_legacy_df.columns if 'url' in col.lower()])
# Drop the "fifa_version" and "fifa_update" columns
males_legacy_df = males_legacy_df.drop(columns=['fifa_version', 'fifa_update'])

In [ ]:
males_legacy_df.isna().sum()

player_id           0
fifa_update_date    0
short_name          0
long_name           0
player_positions    0
                   ..
lcb                 0
cb                  0
rcb                 0
rb                  0
gk                  0
Length: 100, dtype: int64

In [ ]:
males_legacy_df.isnull().sum()

player_id           0
fifa_update_date    0
short_name          0
long_name           0
player_positions    0
                   ..
lcb                 0
cb                  0
rcb                 0
rb                  0
gk                  0
Length: 100, dtype: int64

In [ ]:
# Separating categorical and numerical data
categorical_cols = males_legacy_df.select_dtypes(include=['object']).columns

# Numeric Values
numerical_cols = males_legacy_df.select_dtypes(include=['number'])
numerical_cols = pd.DataFrame(numerical_cols)

In [ ]:
numerical_cols

,player_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,93,95,100500000.0,550000.0,27,169,67,53.0,1.0,...,76,NaN,25,21,20,6,11,15,14,8
1,20801,92,92,79000000.0,375000.0,29,185,80,53.0,1.0,...,85,NaN,22,31,23,7,11,15,14,11
2,9014,90,90,54500000.0,275000.0,30,180,80,19.0,1.0,...,80,NaN,29,26,26,10,8,11,5,15
3,41236,90,90,52500000.0,275000.0,32,195,95,16.0,1.0,...,91,NaN,25,41,27,13,15,10,9,12
4,167495,90,90,63500000.0,300000.0,28,193,92,19.0,1.0,...,37,NaN,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,46,61,110000.0,700.0,18,180,73,2012.0,1.0,...,54,40.0,23,21,25,9,13,13,12,7
161579,269019,46,58,110000.0,750.0,19,188,83,2012.0,1.0,...,31,35.0,50,51,45,6,14,8,13,14
161580,271093,46,58,110000.0,500.0,19,181,73,65.0,1.0,...,37,35.0,36,45,50,8,9,7,14,9
161581,271555,46,70,150000.0,500.0,17,175,68,65.0,1.0,...,63,43.0,19,17,14,13,12,14,7,13


In [ ]:
#categorical values
categorical_cols = males_legacy_df.select_dtypes(include=['object', 'category'])

In [ ]:
categorical_cols

,fifa_update_date,short_name,long_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,nationality_name,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,1987-06-24,La Liga,FC Barcelona,CF,2004-07-01,Argentina,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",1985-02-05,La Liga,Real Madrid CF,LW,2009-07-01,Portugal,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",1984-01-23,Bundesliga,FC Bayern München,SUB,2009-08-28,Netherlands,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,1981-10-03,Ligue 1,Paris Saint-Germain,ST,2012-07-01,Sweden,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,1986-03-27,Bundesliga,FC Bayern München,GK,2011-07-01,Germany,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,2022-09-26,Zhao Ziye,赵梓业,RW,2003-10-01,Super League,Hebei CFFC,RES,2022-05-01,China PR,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,2022-09-26,Wu Junjie,吴俊杰,LB,2003-06-02,Super League,Guangzhou R&F,RES,2022-05-01,China PR,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,2022-09-26,K. Barlow,Kailin Barlow,CDM,2003-06-28,Premier Division,Sligo Rovers,RES,2021-11-02,Republic of Ireland,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",2005-05-20,Premier Division,Shamrock Rovers,RES,2022-06-24,Republic of Ireland,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [ ]:
# Imputation for Numeric Values
imp_numeric = numerical_cols.fillna(numerical_cols.median())

In [ ]:
imp_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 58 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   player_id                       161583 non-null  int64  
 1   overall                         161583 non-null  int64  
 2   potential                       161583 non-null  int64  
 3   value_eur                       161583 non-null  float64
 4   wage_eur                        161583 non-null  float64
 5   age                             161583 non-null  int64  
 6   height_cm                       161583 non-null  int64  
 7   weight_kg                       161583 non-null  int64  
 8   league_id                       161583 non-null  float64
 9   league_level                    161583 non-null  float64
 10  club_team_id                    161583 non-null  float64
 11  club_jersey_number              161583 non-null  float64
 12  club_contract_va

In [ ]:
# Imputation for Categorical Values
imp = SimpleImputer(strategy='most_frequent')
imp_categorical = imp.fit_transform(categorical_cols)
imp_categorical = pd.DataFrame(imp_categorical)

In [ ]:
imp_categorical

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,1987-06-24,La Liga,FC Barcelona,CF,2004-07-01,Argentina,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",1985-02-05,La Liga,Real Madrid CF,LW,2009-07-01,Portugal,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",1984-01-23,Bundesliga,FC Bayern München,SUB,2009-08-28,Netherlands,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,1981-10-03,Ligue 1,Paris Saint-Germain,ST,2012-07-01,Sweden,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,1986-03-27,Bundesliga,FC Bayern München,GK,2011-07-01,Germany,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,2022-09-26,Zhao Ziye,赵梓业,RW,2003-10-01,Super League,Hebei CFFC,RES,2022-05-01,China PR,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,2022-09-26,Wu Junjie,吴俊杰,LB,2003-06-02,Super League,Guangzhou R&F,RES,2022-05-01,China PR,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,2022-09-26,K. Barlow,Kailin Barlow,CDM,2003-06-28,Premier Division,Sligo Rovers,RES,2021-11-02,Republic of Ireland,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",2005-05-20,Premier Division,Shamrock Rovers,RES,2022-06-24,Republic of Ireland,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [ ]:
label_encoder = LabelEncoder()
# Create a new DataFrame to store the encoded values
encoded_df = pd.DataFrame()

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Loop through each column in the DataFrame
for column in imp_categorical.columns:
    # Encode the current column and add it to the new DataFrame
    encoded_df[column] = label_encoder.fit_transform(imp_categorical[column])

# encoded_df contains the integer encoded values of imp_categorical_fifa_21
encoded_df.info()

encoded_df.columns = categorical_cols.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 42 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   0       161583 non-null  int64
 1   1       161583 non-null  int64
 2   2       161583 non-null  int64
 3   3       161583 non-null  int64
 4   4       161583 non-null  int64
 5   5       161583 non-null  int64
 6   6       161583 non-null  int64
 7   7       161583 non-null  int64
 8   8       161583 non-null  int64
 9   9       161583 non-null  int64
 10  10      161583 non-null  int64
 11  11      161583 non-null  int64
 12  12      161583 non-null  int64
 13  13      161583 non-null  int64
 14  14      161583 non-null  int64
 15  15      161583 non-null  int64
 16  16      161583 non-null  int64
 17  17      161583 non-null  int64
 18  18      161583 non-null  int64
 19  19      161583 non-null  int64
 20  20      161583 non-null  int64
 21  21      161583 non-null  int64
 22  22      161583 non-n

In [ ]:
encoded_df

,fifa_update_date,short_name,long_name,player_positions,dob,league_name,club_name,club_position,club_joined_date,nationality_name,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,0,25712,27597,412,2854,15,478,3,48,6,...,216,216,216,204,151,111,111,111,151,25
1,0,8371,8920,1128,2003,15,1013,14,249,134,...,228,228,228,215,170,155,155,155,170,29
2,0,3092,4477,1541,1639,6,481,28,268,117,...,238,238,238,225,156,116,116,116,156,20
3,0,45328,47262,1838,921,26,947,27,659,161,...,249,249,249,195,163,178,178,178,163,34
4,0,29784,29564,719,2409,6,481,5,481,64,...,87,87,87,77,79,82,82,82,79,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,8,45525,49365,1643,8765,40,663,21,3942,34,...,54,54,54,81,78,52,52,52,78,19
161579,8,44350,48141,721,8651,40,633,21,3942,34,...,90,90,90,101,106,127,127,127,106,24
161580,8,22834,25205,289,8675,31,1165,21,3777,137,...,108,108,108,115,118,115,115,115,118,19
161581,8,15941,16771,1694,9156,31,1139,21,3994,137,...,46,46,46,68,61,40,40,40,61,24


In [ ]:
new_males_legacy= pd.concat([imp_numeric, encoded_df], axis=1)

In [ ]:
new_males_legacy

,player_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_level,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023,93,95,100500000.0,550000.0,27,169,67,53.0,1.0,...,216,216,216,204,151,111,111,111,151,25
1,20801,92,92,79000000.0,375000.0,29,185,80,53.0,1.0,...,228,228,228,215,170,155,155,155,170,29
2,9014,90,90,54500000.0,275000.0,30,180,80,19.0,1.0,...,238,238,238,225,156,116,116,116,156,20
3,41236,90,90,52500000.0,275000.0,32,195,95,16.0,1.0,...,249,249,249,195,163,178,178,178,163,34
4,167495,90,90,63500000.0,300000.0,28,193,92,19.0,1.0,...,87,87,87,77,79,82,82,82,79,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,46,61,110000.0,700.0,18,180,73,2012.0,1.0,...,54,54,54,81,78,52,52,52,78,19
161579,269019,46,58,110000.0,750.0,19,188,83,2012.0,1.0,...,90,90,90,101,106,127,127,127,106,24
161580,271093,46,58,110000.0,500.0,19,181,73,65.0,1.0,...,108,108,108,115,118,115,115,115,118,19
161581,271555,46,70,150000.0,500.0,17,175,68,65.0,1.0,...,46,46,46,68,61,40,40,40,61,24


In [ ]:
new_males_legacy.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 100 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   player_id                       161583 non-null  int64  
 1   overall                         161583 non-null  int64  
 2   potential                       161583 non-null  int64  
 3   value_eur                       161583 non-null  float64
 4   wage_eur                        161583 non-null  float64
 5   age                             161583 non-null  int64  
 6   height_cm                       161583 non-null  int64  
 7   weight_kg                       161583 non-null  int64  
 8   league_id                       161583 non-null  float64
 9   league_level                    161583 non-null  float64
 10  club_team_id                    161583 non-null  float64
 11  club_jersey_number              161583 non-null  float64
 12  club_contract_v

2. Feature Engineering

In [ ]:
# Correlation Analysis Model
overall_corr = new_males_legacy.corr()['overall']

In [ ]:
overall_corr

player_id   -0.368772
overall      1.000000
potential    0.695362
value_eur    0.562677
wage_eur     0.607297
               ...   
lcb          0.438253
cb           0.438253
rcb          0.438253
rb           0.498317
gk           0.086577
Name: overall, Length: 100, dtype: float64

In [ ]:
# Correlation threshold (0.5)
relevant_features = [
    'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
    'passing', 'dribbling', 'attacking_short_passing',
    'movement_reactions', 'power_shot_power', 'mentality_vision',
    'mentality_composure'
]

In [ ]:
relevant_features

['potential',
 'value_eur',
 'wage_eur',
 'release_clause_eur',
 'passing',
 'dribbling',
 'attacking_short_passing',
 'movement_reactions',
 'power_shot_power',
 'mentality_vision',
 'mentality_composure']

In [ ]:
# Dropping the columns with low correlation to Overall
filtered_males_legacy = new_males_legacy[relevant_features]

In [ ]:
filtered_males_legacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   potential                161583 non-null  int64  
 1   value_eur                161583 non-null  float64
 2   wage_eur                 161583 non-null  float64
 3   release_clause_eur       161583 non-null  float64
 4   passing                  161583 non-null  float64
 5   dribbling                161583 non-null  float64
 6   attacking_short_passing  161583 non-null  int64  
 7   movement_reactions       161583 non-null  int64  
 8   power_shot_power         161583 non-null  int64  
 9   mentality_vision         161583 non-null  int64  
 10  mentality_composure      161583 non-null  float64
dtypes: float64(6), int64(5)
memory usage: 13.6 MB


In [ ]:
#Scaling the data
sc = StandardScaler()
scaled_males_legacy = sc.fit_transform(filtered_males_legacy)
males_legacy = pd.DataFrame(scaled_males_legacy, columns=filtered_males_legacy.columns)
males_legacy

,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,3.875315,16.447428,24.697104,-0.224202,2.926570,3.480487,2.086204,3.535775,1.499832,2.588430,0.085565
1,3.396013,12.846177,16.681825,-0.224202,2.421008,2.965866,1.609554,3.099571,2.401757,1.957672,0.085565
2,3.076478,8.742427,12.101666,-0.224202,2.623233,3.068791,1.881925,2.990520,1.886371,2.167924,0.085565
3,3.076478,8.407427,12.101666,-0.224202,2.421008,2.451245,1.745740,2.554317,2.337334,2.097840,0.085565
4,3.076478,10.249927,13.246706,-0.224202,0.095424,0.083989,-1.114157,2.990520,-0.948252,-2.317466,0.085565
...,...,...,...,...,...,...,...,...,...,...,...
161578,-1.556774,-0.367900,-0.461712,-0.330265,-1.724598,-1.562798,-1.386528,-2.462022,0.469059,-0.775613,-1.639346
161579,-2.036076,-0.367900,-0.459422,-0.334863,-2.836834,-2.900813,-1.931270,-2.134869,-1.528062,-1.546539,-2.093270
161580,-2.036076,-0.367900,-0.470872,-0.334863,-1.421261,-1.665723,-0.501322,-1.262462,-0.368443,-0.915781,-2.093270
161581,-0.118868,-0.361200,-0.470872,-0.315870,-2.129048,-1.665723,-1.250342,-1.807717,-0.948252,-0.635445,-1.366992


In [ ]:
males_legacy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   potential                161583 non-null  float64
 1   value_eur                161583 non-null  float64
 2   wage_eur                 161583 non-null  float64
 3   release_clause_eur       161583 non-null  float64
 4   passing                  161583 non-null  float64
 5   dribbling                161583 non-null  float64
 6   attacking_short_passing  161583 non-null  float64
 7   movement_reactions       161583 non-null  float64
 8   power_shot_power         161583 non-null  float64
 9   mentality_vision         161583 non-null  float64
 10  mentality_composure      161583 non-null  float64
dtypes: float64(11)
memory usage: 13.6 MB


#3. Training Model

In [ ]:
X = males_legacy
y = new_males_legacy['overall']

In [ ]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

#~Decision Tree

In [ ]:
dt = DecisionTreeRegressor(random_state=48)
dt.fit(Xtrain, ytrain)
dt_y_pred = dt.predict(Xtest)
dt_mae = mean_absolute_error(ytest, dt_y_pred)
rmse = np.sqrt(mean_squared_error(ytest, dt_y_pred))
print("Mean Absolute Error:", dt_mae)
print("Root Mean Square Erroe:", rmse)

Mean Absolute Error: 1.0940990809790512
Root Mean Square Erroe: 1.9018892993098266


#~ Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Assuming Xtrain, ytrain, Xtest, and ytest are already defined

# Step 1: Create polynomial features
degree = 2  # You can choose the degree of the polynomial
poly = PolynomialFeatures(degree=degree)
Xtrain_poly = poly.fit_transform(Xtrain)
Xtest_poly = poly.transform(Xtest)

# Step 2: Train the polynomial regression model
poly_reg = LinearRegression()
poly_reg.fit(Xtrain_poly, ytrain)

# Step 3: Predict the output for the test set
poly_y_pred = poly_reg.predict(Xtest_poly)

# Step 4: Calculate the Mean Absolute Error (MAE)
poly_mae = mean_absolute_error(ytest, poly_y_pred)
print("Mean Absolute Error:", poly_mae)


Mean Absolute Error: 1.7285626276190247


#~xgboost

In [ ]:
xgb_regr = XGBRegressor(
    objective ='reg:squarederror',
    colsample_bytree = 0.3,
    learning_rate = 0.1,
    max_depth = 5, alpha = 10,
    n_estimators = 100,
    random_state=42)

xgb_regr.fit(Xtrain, ytrain)
xgb_y_pred = xgb_regr.predict(Xtest)
xgb_mae = mean_absolute_error(ytest, xgb_y_pred)
print("Mean Absolute Error:", xgb_mae)

Mean Absolute Error: 1.277418222352708


#~Gradient Boost

In [ ]:
gb = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    random_state = 42,
    max_depth = 3)
gb.fit(Xtrain, ytrain)
gb_y_pred = gb.predict(Xtest)
gb_mae = mean_absolute_error(ytest, gb_y_pred)
print("Mean Absolute Error:", gb_mae)

#~Support Vector machine

In [ ]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf')
svr.fit(Xtrain, ytrain)
svr_y_pred = svr.predict(Xtest)
svr_mae = mean_absolute_error(ytest, svr_y_pred)
print("Mean Absolute Error:", svr_mae)

# ***4***. Evaluation


Using Mean Absolute

Fine Tuning the model

Repeated training and testing

Training with GridSearch

In [ ]:
from sklearn.ensemble import RandomForestClassifier
ranf_classifier = RandomForestClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_model = XGBRegressor(n_elements=100,random_state=42)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
}

random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=12,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train, y_train)

print(f"Best parameters for XGBoost: {random_search.best_params_}")
print(f"Best cross-validation R2 score for XGBoost: {random_search.best_score_}")

best_model = random_search.best_estimator_

best_model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:50:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_elements" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters for XGBoost: {'learning_rate': 0.16743239807751675, 'max_depth': 9, 'n_estimators': 269}
Best cross-validation R2 score for XGBoost: 0.9675663539152437


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [23:50:32] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_elements" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
test_path = '/content/drive/My Drive/male_players.csv'

test_data = pd.read_csv(male_players_file_path)

NameError: name 'male_players_file_path' is not defined

# 5. Training and testing with new dataset

In [ ]:
threshold = len(players_22_df) * 0.3

# Drop columns with more than 30% missing values
players22_clean = players_22_df.dropna(thresh=threshold, axis=1)

# Verify the shape of the cleaned dataset
print("Shape of cleaned dataset:", players22_clean.shape)

In [ ]:
columns_to_drop = ["player_url", "club_logo_url", "club_flag_url", "nation_flag_url"]
players22_clean = players22_clean.drop(columns=columns_to_drop)

In [ ]:
players22_clean.head()

In [ ]:
# Separate numeric and non-numeric columns
numeric_cols = players22_clean.select_dtypes(include=[np.number]).columns
non_numeric_cols = players22_clean.select_dtypes(exclude=[np.number]).columns

In [ ]:
# Fill missing values for numeric columns with mean
players22_clean[numeric_cols] = players22_clean[numeric_cols].apply(lambda col: col.fillna(col.mean()), axis=0)

In [ ]:
players22_clean[non_numeric_cols] = players22_clean[non_numeric_cols].apply(lambda col: col.fillna(col.mode()[0]), axis=0)

In [ ]:
print("Shape of cleaned dataset:", players22_clean.shape)

In [ ]:
players22_clean.head()

In [ ]:
features = [
    'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
    'passing', 'dribbling', 'attacking_short_passing',
    'movement_reactions', 'power_shot_power', 'mentality_vision',
    'mentality_composure'
]
Select_interest_features = players_22_df[features]
# Standardize the features
scaler_standard = StandardScaler()
players22_features = scaler_standard.fit_transform(players_22_df[features])
players22_features


In [ ]:
# Categorical columns (if any)
categorical_columns2 = players_22_df.select_dtypes(exclude=['int', 'float']).columns.tolist()

In [ ]:

# Use OneHotEncoder to encode categorical variables
encoder = OneHotEncoder(sparse=False,drop='first')
encoder.fit(players_22_df[categorical_columns2])
encoded_data = encoder.transform(players_22_df[categorical_columns2])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns2))


In [ ]:
# Correlation analysis
numeric_df = players22_clean.select_dtypes(include=[np.number])

# Calculate the correlation matrix
T_correlation = numeric_df.corr()

# Correlation with the target variable 'overall'
correlation_target = abs(T_correlation["overall"])

# Select features with high correlation with 'overall' (0.5 <= correlation < 1)
interest_features = correlation_target[(correlation_target >= 0.5) & (correlation_target < 1)]

# Print the features of interest
print("Features highly correlated with 'overall':")
interest_features

In [ ]:
numeric_df.head()

#Testing

In [ ]:
# Assuming the trained model is saved in a variable called best_estimator
# Use the trained model to make predictions on players22 data
players_predictions = best_rf_classifier.predict(Xtest)

# Calculate MAE and RMSE for the predictions
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae_players22 = mean_absolute_error(ytest, players_predictions)
rmse_players22 = mean_squared_error(ytest, players_predictions, squared=False)

print("Mean Absolute Error:", mae_players22)
print("Root Mean Squared Error:", rmse_players22)

In [ ]:
with open('Fifa_prediction.pkl', 'wb') as model_file:
    pickle.dump(best_estimator, model_file)

In [ ]:
best_estimator.predict(pd.DataFrame(Xtrain.iloc[8]).transpose())